In [ ]:
import os
import pandas as pd
import numpy as np
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import matplotlib.pyplot as plt
import seaborn as sns
import shap

In [ ]:
csv_dir = '../../SEC Trackman Data/'
all_files = [os.path.join(csv_dir, f) for f in os.listdir(csv_dir) if f.endswith('.csv')]

df_list = []

for file in all_files:
    df = pd.read_csv(file)
    df_list.append(df)

all_pitches = pd.concat(df_list, ignore_index=True)

print(all_pitches.shape)
print(all_pitches.columns)

In [ ]:
fc = all_pitches[all_pitches['TaggedPitchType']=='Cutter']
print(fc.shape)

In [ ]:
fc['whiff'] = np.where(fc['PitchCall']=='StrikeSwinging',1,0)

In [ ]:
preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']

In [ ]:
fc_X = fc[preds]
fc_y = fc['whiff']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(fc_X, fc_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
y_pred_prob = model.predict_proba(X_test)[:,1]
roc_auc = roc_auc_score(y_test, predictions, multi_class='ovr')

print(f'ROC AUC: {roc_auc:.4f}')

# Plotting ROC Curve for one class (e.g., class 0)
fpr, tpr, _ = roc_curve(y_test == 1, y_pred_prob)
plt.plot(fpr, tpr, label='Class 0 ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='best')
plt.show()

In [ ]:
aub_fc = fc[fc['PitcherTeam']=='AUB_TIG']

In [ ]:
# Get unique values
unique_values = fc['Pitcher'].unique()
aub_unique_values = aub_fc['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: fc[fc['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: fc[fc['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_fc_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(fc_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} cutter stuff+: {np.mean(stuff * 100)}")

In [ ]:
for player in aub_unique_values:
    aub_fc_stuff(player)

Splits 

In [ ]:
fc_rr = fc[(fc['PitcherThrows'] == 'Right') & (fc['BatterSide'] == 'Right')]
fc_rl = fc[(fc['PitcherThrows'] == 'Right') & (fc['BatterSide'] == 'Left')]
fc_lr = fc[(fc['PitcherThrows'] == 'Left') & (fc['BatterSide'] == 'Right')]
fc_ll = fc[(fc['PitcherThrows'] == 'Left') & (fc['BatterSide'] == 'Left')]

print(fc_rr.shape)
print(fc_rl.shape)
print(fc_lr.shape)
print(fc_ll.shape)

In [ ]:
fc_rr['whiff'] = np.where(fc_rr['PitchCall']=='StrikeSwinging',1,0)
fc_rl['whiff'] = np.where(fc_rl['PitchCall']=='StrikeSwinging',1,0)
fc_lr['whiff'] = np.where(fc_lr['PitchCall']=='StrikeSwinging',1,0)
fc_ll['whiff'] = np.where(fc_ll['PitchCall']=='StrikeSwinging',1,0)

In [ ]:
preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']

In [ ]:
fc_rr_X = fc_rr[preds]
fc_rr_y = fc_rr['whiff']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(fc_rr_X, fc_rr_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
aub_fc_rr = fc_rr[fc_rr['PitcherTeam']=='AUB_TIG']
print(aub_fc_rr.shape)

In [ ]:
# Get unique values
unique_values = fc_rr['Pitcher'].unique()
aub_unique_values = aub_fc_rr['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: fc_rr[fc_rr['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: fc_rr[fc_rr['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_rr_fc_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(fc_rr_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} RR Cutter stuff+: {np.mean(stuff * 100)}")

In [ ]:
for pitcher in aub_unique_values:
    aub_rr_fc_stuff(pitcher)

In [ ]:
fc_rl_X = fc_rl[preds]
fc_rl_y = fc_rl['whiff']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(fc_rl_X, fc_rl_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
aub_fc_rl = fc_rl[fc_rl['PitcherTeam']=='AUB_TIG']
print(aub_fc_rl.shape)

In [ ]:
# Get unique values
unique_values = fc_rl['Pitcher'].unique()
aub_unique_values = aub_fc_rl['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: fc_rl[fc_rl['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: fc_rl[fc_rl['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_rl_fc_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(fc_rl_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} RL Cutter stuff+: {np.mean(stuff * 100)}")

In [ ]:
for pitcher in aub_unique_values:
    aub_rl_fc_stuff(pitcher)

In [ ]:
fc_lr_X = fc_lr[preds]
fc_lr_y = fc_lr['whiff']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(fc_lr_X, fc_lr_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
aub_fc_lr = fc_lr[fc_lr['PitcherTeam']=='AUB_TIG']
print(aub_fc_lr.shape)

In [ ]:
# Get unique values
unique_values = fc_lr['Pitcher'].unique()
aub_unique_values = aub_fc_lr['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: fc_lr[fc_lr['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: fc_lr[fc_lr['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_lr_fc_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(fc_lr_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} LR Cutter stuff+: {np.mean(stuff * 100)}")

In [ ]:
for pitcher in aub_unique_values:
    aub_lr_fc_stuff(pitcher)

In [ ]:
fc_ll_X = fc_ll[preds]
fc_ll_y = fc_ll['whiff']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(fc_ll_X, fc_ll_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
aub_fc_ll = fc_ll[fc_ll['PitcherTeam']=='AUB_TIG']
print(aub_fc_ll.shape)

In [ ]:
# Get unique values
unique_values = fc_ll['Pitcher'].unique()
aub_unique_values = aub_fc_ll['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: fc_ll[fc_ll['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: fc_ll[fc_ll['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_ll_fc_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(fc_ll_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} LL Cutter stuff+: {np.mean(stuff * 100)}")

In [ ]:
for pitcher in aub_unique_values:
    aub_ll_fc_stuff(pitcher)

In [ ]:
aub_subsets['Vail, Tommy']